# pipelines.agent.model

> Agent-friendly wrapper around and LLM instance

In [ ]:
# | default_exp pipelines.agent.model

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import re

from enum import Enum

from smolagents import ChatMessage, Model, get_clean_message_list, tool_role_conversions
from smolagents.models import get_tool_call_from_text, remove_stop_sequences
import onprem

In [ ]:
# | export


class AgentModel(Model):
    """
    A smolagents Model implementation that wraps an onprem LLM instance.
    
    This adapter allows onprem LLM instances to be used with smolagents Agents.
    
    Parameters:
        llm (LLM): An instance of onprem.llm.base.LLM
        **kwargs: Additional keyword arguments to pass to the parent Model class
    """
    
    def __init__(
        self,
        llm: onprem.LLM,
        **kwargs
    ):
        self.llm = llm
        self.maxlength=8192
        self.model_id = self.llm.model_name
        super().__init__(
            flatten_messages_as_text=True,
            **kwargs
        )
        
    def generate(
        self,
        messages,
        stop_sequences=None,
        response_format=None, 
        tools_to_call_from=None,
        **kwargs
        ):
        """
        Process the input messages and return the model's response.
        
        Parameters:
            messages: A list of message dictionaries to be processed.
            stop_sequences: A list of strings that will stop generation if encountered.
            response_format: The response format to use in the model's response.
            tools_to_call_from: A list of tools that the model can use.
            **kwargs: Additional keyword arguments to pass to the LLM.
            
        Returns:
            ChatMessage: A chat message object containing the model's response.
        """
        # Convert smolagents messages to a format that onprem LLM can use
        messages = self.clean(messages)
        
        # Call the LLM with the processed messages
        #import litellm as api
        #result = api.completion(
                #model='ollama_chat/llama3.1:8b',
                #messages=messages,
                #max_tokens=self.maxlength,
                #stream=False,
                #stop=stop_sequences,
            #)
        #data = result["choices"][0]
        #text = data.get("text", data.get("message", data.get("delta")))
        #text = text if isinstance(text, str) else text.get("content")
        #response = text

        response = self.llm.prompt(
            messages,
            stop=stop_sequences or [],
            max_tokens = self.maxlength,
            **kwargs
        )
        #print(f'RESPONSE: {response}')

        # Remove stop sequences from LLM output
        if stop_sequences is not None:
            response = remove_stop_sequences(response, stop_sequences)
        
        # Create and return a ChatMessage with the response
        message =  ChatMessage(role="assistant", content=response)

        # Extract first tool action
        if tools_to_call_from:
            message.tool_calls = [
                get_tool_call_from_text(
                    re.sub(r".*?Action:(.*?\n\}).*", r"\1", response, flags=re.DOTALL), self.tool_name_key, self.tool_arguments_key
                )
            ]

        return message


    def parameters(self, maxlength):
        """
        Set LLM inference parameters.

        Args:
            maxlength: maximum sequence length
        """

        self.maxlength = maxlength


    def clean(self, messages):
        """
        Gets a clean message list.

        Args:
            messages: input messages

        Returns:
            clean messages
        """

        # Get clean message list
        messages = get_clean_message_list(messages, role_conversions=tool_role_conversions, flatten_messages_as_text=self.flatten_messages_as_text)

        # Ensure all roles are strings and not enums for compability across LLM frameworks
        for message in messages:
            if "role" in message:
                message["role"] = message["role"].value if isinstance(message["role"], Enum) else message["role"]

        return messages


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()